In [ ]:
%%capture
!pip install transformers  datasets

In [ ]:
from datasets import *
from transformers import RobertaModel, RobertaTokenizer

import torch
import torch.nn.functional as F

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the JSONL file
df = pd.read_csv('/content/drive/MyDrive/sampled_20000.csv')


In [ ]:
df

,Unnamed: 0,text,labels
0,0,"Hoo boy. A better question might be, how did w...",0
1,1,"There are several formats that could be used,...",0
2,2,Warblington railway station serves the Warblin...,0
3,3,Modern quantum machine learning (QML) method...,0
4,4,The way most people describe audio quality is ...,0
...,...,...,...
39995,39995,\n\nThe royal family is famously known for its...,1
39996,39996,Qianjiang Motorcycle (also known as QJ Motor) ...,1
39997,39997,\n\nWe consider the boundary value problems fo...,1
39998,39998,1520.com is a domain name that was registered ...,1


In [ ]:
# Load the JSONL file
df_val = pd.read_json(r"/content/drive/MyDrive/subtaskA_dev_monolingual.jsonl", lines=True)

In [ ]:
df_val

,text,label,model,source,id
0,Giving gifts should always be enjoyable. Howe...,1,bloomz,wikihow,0
1,Yveltal (Japanese: ユベルタル) is one of the main a...,1,bloomz,wikihow,1
2,If you'd rather not annoy others by being rude...,1,bloomz,wikihow,2
3,If you're interested in visiting gravesite(s) ...,1,bloomz,wikihow,3
4,The following are some tips for becoming succe...,1,bloomz,wikihow,4
...,...,...,...,...,...
4995,The paper deals with an interesting applicatio...,0,human,peerread,4995
4996,This manuscript tries to tackle neural network...,0,human,peerread,4996
4997,The paper introduced a regularization scheme t...,0,human,peerread,4997
4998,Inspired by the analysis on the effect of the ...,0,human,peerread,4998


In [ ]:
df['label']=df['labels'].astype('category')
df['target']=df['label'].cat.codes

df.head()

,Unnamed: 0,text,labels,label,target
0,0,"Hoo boy. A better question might be, how did w...",0,0,0
1,1,"There are several formats that could be used,...",0,0,0
2,2,Warblington railway station serves the Warblin...,0,0,0
3,3,Modern quantum machine learning (QML) method...,0,0,0
4,4,The way most people describe audio quality is ...,0,0,0


In [ ]:
df_val['label']=df_val['label'].astype('category')
df_val['target']=df_val['label'].cat.codes

df_val.head()

,text,label,model,source,id,target
0,Giving gifts should always be enjoyable. Howe...,1,bloomz,wikihow,0,1
1,Yveltal (Japanese: ユベルタル) is one of the main a...,1,bloomz,wikihow,1,1
2,If you'd rather not annoy others by being rude...,1,bloomz,wikihow,2,1
3,If you're interested in visiting gravesite(s) ...,1,bloomz,wikihow,3,1
4,The following are some tips for becoming succe...,1,bloomz,wikihow,4,1


In [ ]:
len(df_val)

5000

In [ ]:
from datasets import *
from transformers import RobertaModel, RobertaTokenizer

import torch
import torch.nn.functional as F

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
df['label'].cat.categories

Index([0, 1], dtype='int64')

In [ ]:
df.shape

(40000, 5)

In [ ]:
category_map = {code: category for code, category in enumerate(df['label'].cat.categories)}
category_map

{0: 0, 1: 1}

In [ ]:
train_end_point = int(df.shape[0])
print(train_end_point)


df_train = df
df_test = df.iloc[35000:,:]
print(df_train.shape, df_test.shape, df_val.shape)

40000
(40000, 5) (5000, 5) (5000, 6)


In [ ]:
from datasets import *

In [ ]:
# Converting pandas DataFrames into Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(df_train.drop('label',axis=1))
# dataset_val = Dataset.from_pandas(df_val.drop('label',axis=1))
dataset_test = Dataset.from_pandas(df_test.drop('label',axis=1))


In [ ]:
from datasets import Dataset, DatasetDict
validation_dataset = Dataset.from_pandas(df_val.drop('label',axis=1))

In [ ]:
validation_dataset

Dataset({
    features: ['text', 'model', 'source', 'id', 'target'],
    num_rows: 5000
})

In [ ]:
dataset_train

Dataset({
    features: ['Unnamed: 0', 'text', 'labels', 'target'],
    num_rows: 40000
})

In [ ]:
# Shuffle the training dataset
dataset_train_shuffled = dataset_train.shuffle(seed=42)  # Using a seed for reproducibility


In [ ]:
# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train_shuffled,
     'val': validation_dataset,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'labels', 'target'],
        num_rows: 40000
    })
    val: Dataset({
        features: ['text', 'model', 'source', 'id', 'target'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'labels', 'target'],
        num_rows: 5000
    })
})

In [ ]:
dataset['train']

Dataset({
    features: ['Unnamed: 0', 'text', 'labels', 'target'],
    num_rows: 40000
})

In [ ]:
df_train.target.value_counts(normalize=True)

,proportion
target,
0,0.5
1,0.5


In [ ]:
class_weights=(1/df_train.target.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights


tensor([0.5000, 0.5000])

In [ ]:
model_name = "FacebookAI/roberta-base"

In [ ]:
ds=dataset

In [ ]:
print(ds['train'][0]['text'])


The binary system was used to denote 210 = 1000 in the original base 10 counting system from the development of the number system. This original counting system was based on fingers and toes and was used in different countries worldwide. In 1671, John Harvard, a cleric at the Massachusetts Bay Colony, established the US currency pound (apothecary weight) as weighing 16 bushel (240 pounds) : 240 ÷ 16 (lb ÷oz) = 15 (pecks) Each peck (or pottle) was 24 pecks of bread (~ 2000) Each bushel of wheat (240 pounds) was 240 ÷ 16 (lb ÷ oz) = 15 (pecks) Therefore, 15 pck = 1 bushel As such, when music was developed in base 10 in the 20th century, the BPM (beats per minute) was based on multiples of 15. In addition, the specific frequencies noted at BPM would seem to be designed for percussion instruments such as drums. If one was to develop a music notation program which perfectly divided the 21st century into 24 equal intervals (sectors) of 11 pcs, then one may see that BPM is close to multiples 

In [ ]:
print(ds['train'][0]['target'])

1


In [ ]:
train_dataset = ds['train'].shuffle(seed=42)
test_dataset = ds['test'].shuffle(seed=42)


In [ ]:
validation_dataset = ds['val']

In [ ]:
validation_dataset

Dataset({
    features: ['text', 'model', 'source', 'id', 'target'],
    num_rows: 5000
})

In [ ]:
from datasets import *
from transformers import RobertaModel, RobertaTokenizer

import torch
import torch.nn.functional as F

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", truncation=True, padding = True, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:

print("Tokenizer max input length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer max input length: 512
Tokenizer vocabulary size: 50265


In [ ]:
def tokenize_text(batch):
  return tokenizer(batch["text"],
                   padding=True,
                   truncation=True,
                   return_token_type_ids=True,
                   max_length=288)

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_text, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_text, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [ ]:
tokenized_val_dataset = validation_dataset.map(tokenize_text, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
tokenized_validation_dataset = tokenized_val_dataset

In [ ]:
print(tokenized_validation_dataset)

Dataset({
    features: ['text', 'model', 'source', 'id', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})


In [ ]:
print(tokenized_train_dataset)

Dataset({
    features: ['Unnamed: 0', 'text', 'labels', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 40000
})


In [ ]:
print(tokenized_test_dataset)

Dataset({
    features: ['Unnamed: 0', 'text', 'labels', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})


In [ ]:
print(tokenized_val_dataset)

Dataset({
    features: ['text', 'model', 'source', 'id', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})


In [ ]:
columns=["target", "input_ids", "attention_mask", "token_type_ids"]

tokenized_train_dataset.set_format("torch", columns=columns)
tokenized_test_dataset.set_format("torch", columns=columns)

In [ ]:
tokenized_val_dataset.set_format("torch", columns=columns)

In [ ]:
tokenized_validation_dataset.set_format("torch", columns=columns)

In [ ]:
BATCH_SIZE = 8

In [ ]:
class MyDataset(Dataset):
  def __init__(self, dataset, partition_key):
    self.dataset = dataset

  def __getitem__(self, index):
    return self.dataset[index]

  def __len__(self):
    return self.dataset.num_rows

In [ ]:
train_data = MyDataset(tokenized_train_dataset, partition_key="train")
test_data = MyDataset(tokenized_test_dataset, partition_key="test")

In [ ]:
val_data = MyDataset(tokenized_val_dataset, partition_key="val")

In [ ]:
validation_data = MyDataset(tokenized_validation_dataset, partition_key="validation")

In [ ]:
from torch.utils.data import DataLoader, Dataset
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE)

In [ ]:
val_loader  = DataLoader(dataset=validation_data, batch_size=BATCH_SIZE)

In [ ]:
validation_loader  = DataLoader(dataset=validation_data, batch_size=BATCH_SIZE)

In [ ]:
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class RobertaWithClassification(torch.nn.Module):
  def __init__(self):
    super(RobertaWithClassification, self).__init__()
    self.roberta = RobertaModel.from_pretrained("roberta-base")
    self.linear = torch.nn.Linear(768, 768)
    self.activation = torch.nn.ReLU()
    self.dropout = torch.nn.Dropout(0.3)
    self.classifier = torch.nn.Linear(768, 2)

  # output of the roberta model:
  # https://huggingface.co/transformers/v3.2.0/main_classes/output.html#basemodeloutputwithpooling
  def forward(self, input_ids, attention_mask, token_type_ids):
    output_with_pooling = self.roberta(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    hidden_state = output_with_pooling[0]
    pooler = hidden_state[:,0]
    pooler = self.linear(pooler)
    pooler = self.activation(pooler)
    pooler = self.dropout(pooler)
    output = self.classifier(pooler)
    return output

In [ ]:
model = RobertaWithClassification()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.to(device)

RobertaWithClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Layer

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
base_param_count = count_parameters(model)
print(base_param_count)

125237762


In [ ]:
import time

In [ ]:
lr = 1e-5
EPOCHS = 2

In [ ]:
def get_accuracy(y_pred, targets):
  predictions = torch.log_softmax(y_pred, dim=1).argmax(dim=1)
  accuracy = (predictions == targets).sum() / len(targets)
  return accuracy



In [ ]:
# prompt: give a function to give f-score  like def get_accuracy(y_pred, targets):
#   predictions = torch.log_softmax(y_pred, dim=1).argmax(dim=1)
#   accuracy = (predictions == targets).sum() / len(targets)
#   return accuracy

from sklearn.metrics import f1_score

def get_f1_score(y_pred, targets, average='macro'):
    predictions = torch.log_softmax(y_pred, dim=1).argmax(dim=1)
    f1 = f1_score(targets.cpu(), predictions.cpu(), average=average)
    return f1


In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
def train(model, train_loader, epochs, optimizer):
  total_time = 0

  for epoch in range(epochs):
    interval = len(train_loader) // 5

    total_train_loss = 0
    total_train_acc = 0



    start = time.time()

    model.train()
    for batch_idx, batch in enumerate(train_loader):
      optimizer.zero_grad()

      input_ids = batch["input_ids"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      token_type_ids = batch["token_type_ids"].to(device)
      labels = batch["target"].to(device)

      outputs = model(input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids)

      loss = loss_function(outputs, labels)
      acc = get_accuracy(outputs, labels)
      f_measure =  get_f1_score(outputs, labels)

      total_train_loss += loss.item()
      total_train_acc += acc.item()


      loss.backward()
      optimizer.step()

      if (batch_idx + 1) % interval == 0:
        print("Batch: %s/%s | Training loss: %.4f | accuracy: %.4f  | fscore: %.4f" % (batch_idx+1, len(train_loader), loss, acc, f_measure ))

    train_loss = total_train_loss / len(train_loader)
    train_acc = total_train_acc / len(train_loader)


    end = time.time()
    hours, remainder = divmod(end - start, 3600)
    minutes, seconds = divmod(remainder, 60)

    print(f"Epoch: {epoch+1} train loss: {train_loss:.4f} train acc: {train_acc:.4f}")
    print("Epoch time elapsed: {:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))
    print("")

    total_time += (end - start)

  # Get the average time per epoch
  average_time_per_epoch = total_time / epochs
  hours, remainder = divmod(average_time_per_epoch, 3600)
  minutes, seconds = divmod(remainder, 60)

  print("Average time per epoch: {:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))

In [ ]:
pip list

In [ ]:
train(model, train_loader, EPOCHS, optimizer)

KeyboardInterrupt: 

In [ ]:
def evaluate(model, test_loader):
  interval = len(test_loader) // 5

  total_test_loss = 0
  total_test_acc = 0

  model.eval()
  with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
      input_ids = batch["input_ids"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      token_type_ids = batch["token_type_ids"].to(device)
      labels = batch["target"].to(device)

      outputs = model(input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids)
      loss = loss_function(outputs, labels)
      acc = get_accuracy(outputs, labels)
      f_measure =  get_f1_score(outputs, labels)

      total_test_loss += loss.item()
      total_test_acc += acc.item()

      if (batch_idx + 1) % interval == 0:
        print("Batch: %s/%s | Test loss: %.4f | accuracy: %.4f | fscore: %.4f" % (batch_idx+1, len(test_loader), loss, acc, f_measure))

  test_loss = total_test_loss / len(test_loader)
  test_acc = total_test_acc / len(test_loader)

  print(f"Test loss: {test_loss:.4f} acc: {test_acc:.4f}")
  print("")

In [ ]:
evaluate(model, test_loader)

KeyboardInterrupt: 

In [ ]:
evaluate(model, val_loader)

Batch: 25/126 | Test loss: 0.0005 | accuracy: 1.0000
Batch: 50/126 | Test loss: 0.0006 | accuracy: 1.0000
Batch: 75/126 | Test loss: 0.0005 | accuracy: 1.0000
Batch: 100/126 | Test loss: 0.0006 | accuracy: 1.0000
Batch: 125/126 | Test loss: 0.0216 | accuracy: 1.0000
Test loss: 0.0577 acc: 0.9851



In [ ]:
evaluate(model, validation_loader)

Batch: 125/625 | Test loss: 0.7226 | accuracy: 0.8750
Batch: 250/625 | Test loss: 1.2079 | accuracy: 0.7500
Batch: 375/625 | Test loss: 0.0005 | accuracy: 1.0000
Batch: 500/625 | Test loss: 0.7280 | accuracy: 0.8750
Batch: 625/625 | Test loss: 2.8985 | accuracy: 0.5000
Test loss: 1.0874 acc: 0.7582



In [ ]:
evaluate(model, test_loader)

Batch: 25/126 | Test loss: 0.0005 | accuracy: 1.0000
Batch: 50/126 | Test loss: 0.7616 | accuracy: 0.8750
Batch: 75/126 | Test loss: 0.0005 | accuracy: 1.0000
Batch: 100/126 | Test loss: 0.0005 | accuracy: 1.0000
Batch: 125/126 | Test loss: 0.0005 | accuracy: 1.0000
Test loss: 0.0501 acc: 0.9851



In [ ]:
evaluate(model, test_loader)

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

In [ ]:
import math

In [ ]:
class LoRALayer(torch.nn.Module):
  def __init__(self, in_dim, out_dim, r, alpha):
    super().__init__()
    self.r = r
    self.alpha = alpha

    # Initialize A to kaiming uniform following code: https://github.com/microsoft/LoRA/blob/main/loralib/layers.py
    self.A = torch.nn.Parameter(torch.empty(r, in_dim))
    # Initialize B to zeros.
    self.B = torch.nn.Parameter(torch.empty(out_dim, r))
    torch.nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
    torch.nn.init.zeros_(self.B)

    self.scaling = self.alpha / self.r

  def forward(self, x):
    x = self.scaling * (x @ self.A.transpose(0, 1) @ self.B.transpose(0, 1))
    return x

In [ ]:
class LinearWithLoRA(torch.nn.Module):
  def __init__(self, linear, r, alpha):
    super().__init__()
    self.linear = linear
    self.lora = LoRALayer(
        linear.in_features, linear.out_features, r, alpha
    )

  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [ ]:
lora_model = RobertaWithClassification()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for param in lora_model.parameters():
  param.requires_grad = False

In [ ]:
from functools import partial

In [ ]:
lora_r = 16
lora_alpha = lora_r * 2

assign_lora = partial(LinearWithLoRA, r=lora_r, alpha=lora_alpha)

In [ ]:
print(lora_model)

RobertaWithClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Layer

In [ ]:
import math
for layer in lora_model.roberta.encoder.layer:
  layer.attention.self.query = assign_lora(layer.attention.self.query)
  layer.attention.self.value = assign_lora(layer.attention.self.value)

In [ ]:
lora_param_count = count_parameters(lora_model)
print("Model with LoRA param count:", lora_param_count)
print("Base model param count:", base_param_count)
print(str(base_param_count // lora_param_count) + " times smaller than base model")

Model with LoRA param count: 589824
Base model param count: 125237762
212 times smaller than base model


In [ ]:
lora_model.to(device)

RobertaWithClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): LinearWithLoRA(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora): LoRALayer()
              )
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): LinearWithLoRA(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora): LoRALayer()
              )
              (dro

In [ ]:
lr = 1e-5
EPOCHS = 2

In [ ]:
optimizer_lora = torch.optim.Adam(params=lora_model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
train(lora_model, train_loader, EPOCHS, optimizer_lora)

Batch: 1000/5000 | Training loss: 0.5689 | accuracy: 0.6250  | fscore: 0.5636
Batch: 2000/5000 | Training loss: 0.1591 | accuracy: 1.0000  | fscore: 1.0000
Batch: 3000/5000 | Training loss: 0.0833 | accuracy: 1.0000  | fscore: 1.0000
Batch: 4000/5000 | Training loss: 0.0482 | accuracy: 1.0000  | fscore: 1.0000
Batch: 5000/5000 | Training loss: 0.2232 | accuracy: 0.8750  | fscore: 0.8730
Epoch: 1 train loss: 0.3225 train acc: 0.8605
Epoch time elapsed: 00:52:45.41

Batch: 1000/5000 | Training loss: 0.3078 | accuracy: 0.8750  | fscore: 0.8545
Batch: 2000/5000 | Training loss: 0.0359 | accuracy: 1.0000  | fscore: 1.0000
Batch: 3000/5000 | Training loss: 0.0280 | accuracy: 1.0000  | fscore: 1.0000
Batch: 4000/5000 | Training loss: 0.0224 | accuracy: 1.0000  | fscore: 1.0000
Batch: 5000/5000 | Training loss: 0.0185 | accuracy: 1.0000  | fscore: 1.0000
Epoch: 2 train loss: 0.0787 train acc: 0.9780
Epoch time elapsed: 00:53:03.01

Average time per epoch: 00:52:54.21


In [ ]:

evaluate(lora_model, validation_loader)

Batch: 125/625 | Test loss: 0.5694 | accuracy: 0.8750 | fscore: 0.4667
Batch: 250/625 | Test loss: 0.0242 | accuracy: 1.0000 | fscore: 1.0000
Batch: 375/625 | Test loss: 0.5221 | accuracy: 0.7500 | fscore: 0.4286
Batch: 500/625 | Test loss: 0.0189 | accuracy: 1.0000 | fscore: 1.0000
Batch: 625/625 | Test loss: 0.0605 | accuracy: 1.0000 | fscore: 1.0000
Test loss: 0.4328 acc: 0.8498



In [ ]:
evaluate(model, validation_loader)

KeyboardInterrupt: 

In [ ]:
validation_loader

In [ ]:
validation_data

AttributeError: 'MyDataset' object has no attribute '_info'

In [ ]:

evaluate(lora_model, test_loader)

Batch: 125/625 | Test loss: 0.0982 | accuracy: 0.8750 | fscore: 0.4667
Batch: 250/625 | Test loss: 0.0109 | accuracy: 1.0000 | fscore: 1.0000
Batch: 375/625 | Test loss: 0.0113 | accuracy: 1.0000 | fscore: 1.0000
Batch: 500/625 | Test loss: 0.0112 | accuracy: 1.0000 | fscore: 1.0000
Batch: 625/625 | Test loss: 0.0111 | accuracy: 1.0000 | fscore: 1.0000
Test loss: 0.0252 acc: 0.9948



In [ ]:
validation_data

AttributeError: 'MyDataset' object has no attribute '_info'

In [ ]:
tokenized_validation_dataset

Dataset({
    features: ['text', 'model', 'source', 'id', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})